In [1]:
!pip install datasets
from datasets import load_dataset

dataset = load_dataset("copenlu/answerable_tydiqa")
train_set = dataset["train"]
validation_set = dataset["validation"]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 12.8 MB/s eta 0:00:00


Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/116067 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/13325 [00:00<?, ? examples/s]

In [2]:
import torch
device = torch.device("cpu")
if torch.cuda.is_available():
  device = torch.device("cuda")
device

device(type='cuda')

In [3]:
languages = ['arabic', 'bengali', 'indonesian']
train_sets = {lang: train_set.filter(lambda example: example['language'] == lang) for lang in languages}
validation_sets = {lang: validation_set.filter(lambda example: example['language'] == lang) for lang in languages}

Filter:   0%|          | 0/116067 [00:00<?, ? examples/s]

Filter:   0%|          | 0/116067 [00:00<?, ? examples/s]

Filter:   0%|          | 0/116067 [00:00<?, ? examples/s]

Filter:   0%|          | 0/13325 [00:00<?, ? examples/s]

Filter:   0%|          | 0/13325 [00:00<?, ? examples/s]

Filter:   0%|          | 0/13325 [00:00<?, ? examples/s]

In [4]:
arabic_vocab_text = ['<pad>'] + sorted(set([token for sent in train_sets['arabic']['document_plaintext'] for token in sent.split()]))
arabic_vocab_question = ['<pad>'] + sorted(set([token for sent in train_sets['arabic']['question_text'] for token in sent.split()]))
bengali_vocab_text = ['<pad>'] + sorted(set([token for sent in train_sets['bengali']['document_plaintext'] for token in sent.split()]))
bengali_vocab_question = ['<pad>'] + sorted(set([token for sent in train_sets['bengali']['question_text'] for token in sent.split()]))
indonesian_vocab_text = ['<pad>'] + sorted(set([token for sent in train_sets['indonesian']['document_plaintext'] for token in sent.split()]))
indonesian_vocab_question = ['<pad>'] + sorted(set([token for sent in train_sets['indonesian']['question_text'] for token in sent.split()]))


In [5]:
import re
import string

def custom_standardization(input_data):
    lowercase = input_data.lower()
    lowercase = re.sub('\[\d+\]', '', lowercase)
    lowercase = re.sub('\n', ' ', lowercase)
    #lowercase = re.sub('[%s]' % re.escape(string.punctuation), '', lowercase)
    return lowercase

In [6]:
arabic_text_sentences = [custom_standardization(text) for text in train_sets['arabic']['document_plaintext']]

In [ ]:
!pip install transformers
import transformers
from transformers import AutoTokenizer
from torch.utils.data import Dataset
from transformers import AutoModelForCausalLM, AutoModelForSequenceClassification
from transformers import Trainer, TrainingArguments
import math


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 58.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 75.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 72.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 30.2 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.18.0
    Uninstalling huggingface-hub-0.18.0:
      Successfully uninstalled huggingface-hub-0.18.0


In [ ]:
model_checkpoint = "distilgpt2"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)
model = AutoModelForCausalLM.from_pretrained(model_checkpoint)

In [ ]:
!pip install BPEmb

In [ ]:
from bpemb import BPEmb

bpemb_ar = BPEmb(lang="ar", dim=300)

downloading https://nlp.h-its.org/bpemb/ar/ar.wiki.bpe.vs10000.model


100%|██████████| 428120/428120 [00:00<00:00, 948303.31B/s]


downloading https://nlp.h-its.org/bpemb/ar/ar.wiki.bpe.vs10000.d300.w2v.bin.tar.gz


100%|██████████| 11198290/11198290 [00:01<00:00, 9000343.88B/s]


In [ ]:


def tokenizer(text):
  return {'input_ids': bpemb_ar.encode_ids_with_eos(text)}

def tokenize_function(examples):
    return tokenizer(examples)

def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
    # customize this part to your needs.
    total_length = (total_length // block_size) * block_size
    # Split by chunks of size block_size .
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    return result

block_size = 128

train_sets_splits=train_sets['arabic']['document_plaintext'].train_test_split(test_size=0.01)
tokenized_datasets = train_sets['arabic']['document_plaintext'].map(tokenize_function, batched=True, num_proc=4, remove_columns=["text"])
print(tokenized_datasets['train'][3]['input_ids'][:10])

lm_datasets = tokenized_datasets.map(group_texts, batched=True, batch_size=1000, num_proc=4,)

AttributeError: ignored